In [24]:
# Section 1: Imports & Model Loading
# =============================================================================
import tkinter as tk
from tkinter import filedialog, Label, Button
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from PIL import Image, ImageTk

# Load the trained model
model = load_model("my_project_name.keras")  # Update with your model file path
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Define age bins used in training (adjust if needed)
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

In [25]:
# Section 2: Image Upload Prediction Functionality
# =============================================================================
def preprocess_image(image_path):
    """Load and preprocess image from a file path for prediction."""
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))  # Match model input size
    image = np.array(image, dtype="float32") / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def predict_image_age():
    """Upload image, predict age range, and display result."""
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if not file_path:
        return
    processed_image = preprocess_image(file_path)
    predictions = model.predict(processed_image)
    predicted_bin = np.argmax(predictions)
    if predicted_bin == len(age_bins) - 1:
        age_range = f"{age_bins[predicted_bin]}+ years"
    else:
        age_range = f"{age_bins[predicted_bin]} - {age_bins[predicted_bin + 1]} years"
    result_label.config(text=f"Predicted Age Range: {age_range}")
    
    # Display the image in the GUI
    img = Image.open(file_path)
    img = img.resize((200, 200))
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.photo = img_tk

In [26]:
# Section 3: Live Camera Prediction Functionality
# =============================================================================
# Global variables for camera control
cap = None
camera_running = False

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def update_frame():
    """Capture and process video frames, detect faces, and predict age."""
    global cap, camera_running, age_bins, model
    if camera_running:
        ret, frame = cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
            
            for (x, y, w, h) in faces:
                # Draw green rectangle around the face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Crop and preprocess the face region
                face_roi = frame[y:y+h, x:x+w]
                try:
                    face_roi = cv2.resize(face_roi, (128, 128))
                    face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
                    face_roi = np.array(face_roi, dtype="float32") / 255.0
                    face_roi = np.expand_dims(face_roi, axis=0)
                    
                    # Predict age range for the face
                    predictions = model.predict(face_roi)
                    predicted_bin = np.argmax(predictions)
                    if predicted_bin == len(age_bins) - 1:
                        age_range = f"{age_bins[predicted_bin]}+ years"
                    else:
                        age_range = f"{age_bins[predicted_bin]} - {age_bins[predicted_bin+1]} years"
                    
                    # Display the predicted age under the face box
                    cv2.putText(frame, age_range, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX,
                                0.6, (0,255,0), 2)
                except Exception as e:
                    print("Error processing face:", e)
            
            # Convert frame for Tkinter display
            cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(cv2image)
            img_pil = img_pil.resize((400, 300))
            imgtk = ImageTk.PhotoImage(image=img_pil)
            image_label.config(image=imgtk)
            image_label.photo = imgtk
        
        # Schedule the next frame update
        root.after(30, update_frame)

def start_camera():
    """Start the live camera feed."""
    global cap, camera_running
    cap = cv2.VideoCapture(0)
    camera_running = True
    update_frame()

def stop_camera():
    """Stop the live camera feed."""
    global camera_running, cap
    camera_running = False
    if cap is not None:
        cap.release()
    image_label.config(image="")

In [29]:
# Section 4: GUI Layout and Widgets
# =============================================================================
root = tk.Tk()
root.title("Age Detection GUI")
root.geometry("450x600")

# Title Label
title_label = Label(root, text="Age Detection with MobileNetV2", font=("Helvetica", 16))
title_label.pack(pady=10)

# Button: Upload Image for Prediction
upload_button = Button(root, text="Upload Image and Predict", command=predict_image_age, font=("Helvetica", 14))
upload_button.pack(pady=10)

# Button: Start Camera
start_button = Button(root, text="Start Camera", command=start_camera, font=("Helvetica", 14))
start_button.pack(pady=10)

# Button: Stop Camera
stop_button = Button(root, text="Stop Camera", command=stop_camera, font=("Helvetica", 14))
stop_button.pack(pady=10)

# Label to display prediction result
result_label = Label(root, text="", font=("Helvetica", 14))
result_label.pack(pady=10)

# Label to display images/video feed
image_label = Label(root)
image_label.pack(pady=10)

In [30]:
# =============================================================================
# Section 5: Start the Tkinter Main Loop
# =============================================================================
root.mainloop()